In [1]:
# Import libraries
import pandas as pd
import re
import requests as req
from bs4 import BeautifulSoup as bs
import json

# Importar datos

In [2]:
# Import data
df=pd.read_csv("../data/Artworks.csv")

df.head()

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


# Evaluación de datos

In [3]:
artist = list(df.Artist.unique())

In [4]:
len(artist)

13863

In [5]:
df.shape

(132403, 29)

In [6]:
df.isnull().sum()

Title                     45
Artist                  1461
ConstituentID           1461
ArtistBio               4734
Nationality             1461
BeginDate               1461
EndDate                 1461
Gender                  1461
Date                    2166
Medium                 11225
Dimensions             11243
CreditLine              2924
AccessionNumber            0
Classification             0
Department                 0
DateAcquired            5994
Cataloged                  0
ObjectID                   0
URL                    57280
ThumbnailURL           68263
Circumference (cm)    132393
Depth (cm)            120479
Diameter (cm)         130985
Height (cm)            18765
Length (cm)           131665
Weight (kg)           132110
Width (cm)             19664
Seat Height (cm)      132403
Duration (sec.)       129258
dtype: int64

Trabajo con los artistas que han presentado trabajos en los últimos 50 años.

# CLEANING

In [7]:
# DROP columns that are not useful for my analysis.

df1 = df.drop(["Title", "ConstituentID", "ArtistBio","Medium","ThumbnailURL", "Circumference (cm)", "Depth (cm)", "Diameter (cm)", "Height (cm)", "Length (cm)", "Weight (kg)", "Width (cm)", "Seat Height (cm)", "Duration (sec.)", "Dimensions", "CreditLine", "AccessionNumber", "Department", "DateAcquired", "Cataloged", "ObjectID", "URL"],axis=1)

In [8]:
df1.sample()

,Artist,Nationality,BeginDate,EndDate,Gender,Date,Classification
33432,Diego Rivera,(Mexican),(1886),(1957),(Male),1926,Drawing


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132403 entries, 0 to 132402
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Artist          130942 non-null  object
 1   Nationality     130942 non-null  object
 2   BeginDate       130942 non-null  object
 3   EndDate         130942 non-null  object
 4   Gender          130942 non-null  object
 5   Date            130237 non-null  object
 6   Classification  132403 non-null  object
dtypes: object(7)
memory usage: 7.1+ MB


In [10]:
date = list(df1.Date.unique())

In [11]:
date

['1896',
 '1987',
 '1903',
 '1980',
 '1976-77',
 '1968',
 '1900',
 '1978',
 '1905',
 '1906',
 '1979',
 '1980-81',
 '1918',
 '1970',
 '1975',
 '1984',
 '1986',
 '1974',
 'n.d.',
 'c. 1917',
 '1917',
 '1923',
 'Unknown',
 '1930',
 '1936',
 '1935',
 '1937',
 '1938',
 '1977',
 '1958',
 '1985',
 '1989',
 '1949',
 '1958–1964',
 'c. 1935',
 '1991',
 '1941',
 '1965',
 '1981',
 '1983',
 '1985–1988',
 'c. 1989-91',
 '1992',
 '1915-17',
 '1915–1917',
 'c. 1915-17',
 '1953',
 '1910',
 'c.1985',
 '1982–1986',
 '1982-86',
 '1945',
 '1985.',
 '1986.',
 '1923–1924',
 '.1-3 1987; .4 1990',
 '1990',
 '1976',
 '1995',
 '1927–1931',
 'c. 1929-30',
 '1964',
 '1959',
 'c. 1918-20',
 'c.1918-1920',
 '1939',
 'c.1976',
 '1975-79',
 '1993',
 '1989.',
 '1996',
 '1988',
 '1982-83',
 '1982–1983',
 '1952-53',
 '1921',
 '1957',
 '1972',
 '1956-57',
 '1924',
 '1962',
 '1925',
 '1960',
 '1969',
 '1963',
 '1994',
 ' 1961',
 '1960-61',
 '1952',
 'c. 1978-84',
 '1927',
 '1979–1985',
 'before 1933',
 '1961',
 '1929',
 'c

In [12]:
df1['year'] = df1['Date'].str.extract(r"(\d+)")

In [13]:
df1.head()

,Artist,Nationality,BeginDate,EndDate,Gender,Date,Classification,year
0,Otto Wagner,(Austrian),(1841),(1918),(Male),1896,Architecture,1896
1,Christian de Portzamparc,(French),(1944),(0),(Male),1987,Architecture,1987
2,Emil Hoppe,(Austrian),(1876),(1957),(Male),1903,Architecture,1903
3,Bernard Tschumi,(),(1944),(0),(Male),1980,Architecture,1980
4,Emil Hoppe,(Austrian),(1876),(1957),(Male),1903,Architecture,1903


In [14]:
df1 = df1.drop(["Date"],axis=1)

In [15]:
df1.sample()

,Artist,Nationality,BeginDate,EndDate,Gender,Classification,year
104024,Mateo López,(Colombian),(1978),(0),(Male),Installation,2008


In [16]:
# Elimino los NaN en la columna year.
df1 = df1[df1['year'].notna()]

In [17]:
df1['year'] = df1['year'].astype(int)

In [18]:
# Me centro en 50 últimos años.
df2 = df1[df1["year"]>=1972]

In [19]:
df2.shape

(42319, 7)

In [20]:
# Elimino los NaN en la columna artist.
df2 = df2[df2['Artist'].notna()]

In [21]:
artist = list(df2.Artist.unique())

In [22]:
artist

['Christian de Portzamparc',
 'Bernard Tschumi',
 'Peter Eisenman, Robert Cole',
 'Rem Koolhaas, Madelon Vriesendorp',
 'Roger C. Ferri',
 'Michael Graves',
 'Aldo Rossi, Gianni Braghieri, M. Bosshard',
 'Steven Holl',
 'Paul Rudolph',
 'Peter Cook',
 'Diller + Scofidio, Elizabeth Diller, Ricardo Scofidio',
 'Zaha Hadid',
 'Fumihiko Maki',
 'Rem Koolhaas, Zoe Zenghelis, Elia Zenghelis, Madelon Vriesendorp',
 'Mario Bellini',
 'Venturi and Rauch, Robert Venturi, John Rauch, Denise Scott Brown',
 'Venturi, Rauch and Scott Brown, Robert Venturi, John Rauch, Denise Scott Brown',
 'Emilio Ambasz',
 'Raimund Abraham',
 'Tadao Ando',
 'Arata Isozaki',
 'Neil M. Denari',
 'Mario Botta',
 'Alessandro Mendini',
 'Leon Krier',
 'Daniel Libeskind',
 'Thom Mayne, Andrew Zago',
 'Morphosis, Santa Monica, CA, Thom Mayne, Andrew Zago',
 'Rem Koolhaas, Elia Zenghelis, Zoe Zenghelis, Madelon Vriesendorp',
 'Aldo Rossi',
 'Joel Sanders',
 'Massimo Scolari',
 'Richard Meier',
 'Rafael Viñoly',
 'Santiago 

In [23]:
#LIMPIEZA columna Artist.
df2['artist_2'] = df2['Artist'].str.split(",")

In [24]:
df2.sample()

,Artist,Nationality,BeginDate,EndDate,Gender,Classification,year,artist_2
52122,William Christenberry,(American),(1936),(2016),(Male),Photograph,1979,[William Christenberry]


In [25]:
art = list(df2.artist_2)

In [26]:
art

[['Christian de Portzamparc'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 

In [27]:
df2['artist_2'] = df2['Artist'].str.split(",")

In [28]:
df2['country'] = df2['Nationality'].str.split(",").replace("()", "")

In [29]:
df2

,Artist,Nationality,BeginDate,EndDate,Gender,Classification,year,artist_2,country
1,Christian de Portzamparc,(French),(1944),(0),(Male),Architecture,1987,[Christian de Portzamparc],[(French)]
3,Bernard Tschumi,(),(1944),(0),(Male),Architecture,1980,[Bernard Tschumi],[()]
5,Bernard Tschumi,(),(1944),(0),(Male),Architecture,1976,[Bernard Tschumi],[()]
6,Bernard Tschumi,(),(1944),(0),(Male),Architecture,1976,[Bernard Tschumi],[()]
7,Bernard Tschumi,(),(1944),(0),(Male),Architecture,1976,[Bernard Tschumi],[()]
...,...,...,...,...,...,...,...,...,...
132308,Christian Vinck Henriquez,(Venezuelan),(1978),(0),(),Print,2016,[Christian Vinck Henriquez],[(Venezuelan)]
132309,Christian Vinck Henriquez,(Venezuelan),(1978),(0),(),Multiple,2016,[Christian Vinck Henriquez],[(Venezuelan)]
132331,Laura Owens,(American),(1970),(0),(Female),Illustrated Book,2015,[Laura Owens],[(American)]
132332,Laura Owens,(American),(1970),(0),(Female),Illustrated Book,2015,[Laura Owens],[(American)]


In [31]:
#LIMPIEZA columna country.
count = list(df2.country)
count

[['(French)'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['(American) ()'],
 ['(Dutch) (Dutch)'],
 ['(American)'],
 ['()'],
 ['(American)'],
 ['()'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(Italian) (Italian) (Italian)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(British)'],
 ['(American) (Americ

In [32]:
df2 = df2[df2['Artist'].notna()]

In [33]:
df2 = df2[df2['Artist'].notna()]

In [34]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42177 entries, 1 to 132399
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Artist          42177 non-null  object
 1   Nationality     42177 non-null  object
 2   BeginDate       42177 non-null  object
 3   EndDate         42177 non-null  object
 4   Gender          42177 non-null  object
 5   Classification  42177 non-null  object
 6   year            42177 non-null  int64 
 7   artist_2        42177 non-null  object
 8   country         42177 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.2+ MB


In [35]:
df3 = df2.drop(df2[df2['Nationality']=="()"].index)

In [36]:
df3

,Artist,Nationality,BeginDate,EndDate,Gender,Classification,year,artist_2,country
1,Christian de Portzamparc,(French),(1944),(0),(Male),Architecture,1987,[Christian de Portzamparc],[(French)]
65,"Peter Eisenman, Robert Cole",(American) (),(1932) (0),(0) (0),(Male) (Male),Architecture,1975,"[Peter Eisenman, Robert Cole]",[(American) ()]
66,"Rem Koolhaas, Madelon Vriesendorp",(Dutch) (Dutch),(1944) (1945),(0) (0),(Male) (Female),Architecture,1987,"[Rem Koolhaas, Madelon Vriesendorp]",[(Dutch) (Dutch)]
67,Roger C. Ferri,(American),(1949),(1991),(Male),Architecture,1979,[Roger C. Ferri],[(American)]
69,Roger C. Ferri,(American),(1949),(1991),(Male),Architecture,1979,[Roger C. Ferri],[(American)]
...,...,...,...,...,...,...,...,...,...
132308,Christian Vinck Henriquez,(Venezuelan),(1978),(0),(),Print,2016,[Christian Vinck Henriquez],[(Venezuelan)]
132309,Christian Vinck Henriquez,(Venezuelan),(1978),(0),(),Multiple,2016,[Christian Vinck Henriquez],[(Venezuelan)]
132331,Laura Owens,(American),(1970),(0),(Female),Illustrated Book,2015,[Laura Owens],[(American)]
132332,Laura Owens,(American),(1970),(0),(Female),Illustrated Book,2015,[Laura Owens],[(American)]


In [37]:
count = list(df3.country)
count

[['(French)'],
 ['(American) ()'],
 ['(Dutch) (Dutch)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(Italian) (Italian) (Italian)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(British)'],
 ['(American) (American) (American)'],
 ['(British)'],
 ['(Japanese)'],
 ['(Dutch) (British) (British) (Dutch)'],
 ['(Italian)'],
 ['(American) (American) (American) (American)'],
 ['(American) (American) (American) (American)'],
 ['(American) (American) (American) (American)'],
 ['(American) (American) (American) (American)'],
 ['(Argentine)'],
 ['(American)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],
 ['(Japanese)'],

# Enrich data base with scraping

EDUCATIONAL DATA (Creativity index)

In [38]:
url = 'http://chartsbin.com/view/41109'

In [39]:
html=req.get(url).content

In [40]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [41]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [42]:
filas=tabla.find_all('tr')

elementos=[[e.text for e in fila] for fila in filas]

elementos[:5]

[['Rank',
  'Country',
  'Technology',
  'Talent',
  'Tolerance',
  'Global Creativity Index'],
 ['1', 'Australia', '7', '1', '4', '0.97'],
 ['2', 'United States', '4', '3', '11', '0.95'],
 ['3', 'New Zealand', '7', '8', '3', '0.949'],
 ['4', 'Canada', '13', '14', '1', '0.92']]

ECONOMY

In [43]:
url = 'https://datosmacro.expansion.com/pib?anio=1999'

In [44]:
html=req.get(url).content

In [45]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [46]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [51]:
filas=tabla.find_all('tr')

elementos=[[e.text.replace("\xa0M€", "").replace("\xa0M$", "").replace("[+]", "").strip() for e in fila] for fila in filas]

elementos[:5]

[['Países', 'Fecha', 'PIB anual', 'PIB anual', 'Var. PIB (%)'],
 ['España', '1999', '595.723', '', '635.968', '', '4,5%', ''],
 ['Alemania', '1999', '2.059.480', '', '2.197.130', '', '1,9%', ''],
 ['Reino Unido', '1999', '1.578.395', '', '1.682.590', '', '3,3%', ''],
 ['Francia', '1999', '1.400.999', '', '1.492.670', '', '3,4%', '']]